<a href="https://colab.research.google.com/github/dinoelT/TestScore-dataViz/blob/main/testScore_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import altair as alt
import pandas as pd
import requests
import codecs
from requests.exceptions import HTTPError
import openpyxl

In [2]:
#Add export?format=xlsx after id
url = "https://docs.google.com/spreadsheets/d/1pUDOO5rROKI20sP5p5_2gVCMvsOoBoFzYUNtz2fZYqA/export?format=xlsx"

In [3]:

try:
  #Send get request
  response = requests.get(url)
  # If the response was successful, no Exception will be raised
  response.raise_for_status()
except HTTPError as http_err:
  print(f'HTTP error occurred: {http_err}')
except Exception as err:
  print(f'Other error occurred: {err}')
else:
  print('Request successfully!')

Request successfully!


In [4]:
excelFileName = "excel.xlsx"

writeSuccessful = True
try:
  with codecs.open(excelFileName, 'wb') as f:
    f.write(response.content)
except Exception as err:
  print(f"Error occured when writing file: {err}")
  writeSuccessful = False
else:
  print("Excel file written successfully")


Excel file written successfully


In [5]:
#Get the sheet names in excel. 
sheetNames = openpyxl.load_workbook(excelFileName, read_only=True).sheetnames
sheetNamesDict = {s:'' for s in sheetNames}
print(f"sheetNamesDict = {sheetNamesDict}")

sheetNamesDict = {'ИТБ': '', 'ФЭТ': '', 'ФМ': ''}


In [6]:
#Altair does not work with russian characters. Define sheet names in english
sheetNamesDict = {'ИТБ': 'ITMB', 'ФЭТ': 'FET', 'ФМ': 'FM'}

In [7]:
if writeSuccessful:
  dataSource = excelFileName
else:
  dataSource = response.content

#Read pandas dataframes from Excel file that was saved
dfs = []
for sheetNameRu in sheetNamesDict:
  df = pd.read_excel(dataSource, header=1, sheet_name=sheetNameRu)
  sheetNameEn = sheetNamesDict[sheetNameRu]
  df['program'] = sheetNameEn
  dfs.append(df)

In [8]:
#Concatenate the dataframes from all sheets
df = pd.concat(dfs, ignore_index=True)

#Drop the columns which have all nan values
df = df.dropna(axis='columns', how = 'all')

In [9]:
columnNamesDict = {col : '' for col in list(df.columns)} 
print(f"columnNamesDict = {columnNamesDict}")

columnNamesDict = {'№': '', 'ФИО': '', 'сумма баллов': '', 'Мат.': '', 'Русс.': '', 'ИКТ': '', 'Бюджет/договор': '', 'program': '', 'Ин.яз': ''}


In [10]:
#Remove the columns that you don't need and provide the english translation for the columns left
#columnNamesDict = {'№': '', 'ФИО': '', 'сумма баллов': '', 'Мат.': '', 'Русс.': '', 'ИКТ': '', 'Бюджет/договор': '', 'program': '', 'Ин.яз': ''}
columnNamesDict = {'ФИО': 'name', 'сумма баллов': 'total_score', 'Мат.': 'ege_math', 'Русс.': 'ege_russian', 'ИКТ': 'ege_it', 'Бюджет/договор': 'is_budget_placement', 'program': 'program', 'Ин.яз': 'ege_foreign'}

In [11]:
#Filter only the needed columns
df = df[columnNamesDict.keys()]

#Replace the russian column names with the english stored in the dictionary
columnNamesEn = [columnNamesDict[col_ru] for col_ru in df.columns]
df.columns = columnNamesEn

In [12]:
#Distinct values in is_budget_placement column
print(f"Distinct values in is_budget_placement column: {df.is_budget_placement.unique()}")

#Replace 'договор' with False
df.is_budget_placement.where(df.is_budget_placement != 'договор', False, inplace=True)
#Replace 'Бюджет' with True
df.is_budget_placement.where(df.is_budget_placement != 'Бюджет', True, inplace=True)
#Replace Nan values with False
df.is_budget_placement.fillna(value = False, inplace=True)

Distinct values in is_budget_placement column: ['договор' 'Бюджет' nan]


In [13]:
df.head()

,name,total_score,ege_math,ege_russian,ege_it,is_budget_placement,program,ege_foreign
0,Абрамов Петр Александрович,243,75.0,98.0,70.0,False,ITMB,NaN
1,Амерханова Наргиза Артуровна,286,96.0,89.0,96.0,True,ITMB,NaN
2,Баранов Алексей Алексеевич,235,45.0,96.0,94.0,False,ITMB,NaN
3,Баубеков Мирас Булатулы,217,70.0,81.0,66.0,False,ITMB,NaN
4,Богун Владислав Олегович,242,78.0,85.0,79.0,False,ITMB,NaN


In [14]:
x_col_name = 'ege_math'
y_col_name = 'ege_russian'

alt.Chart(df).mark_circle(size=100).encode(
    alt.X(x_col_name, scale=alt.Scale(zero=False)),
    alt.Y(y_col_name,scale=alt.Scale(zero=False)),
    color='program',
    tooltip=['name', 'total_score']
).interactive()

alt.Chart(...)